# Vehicle Detection and Tracking: Challenge
---

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
from skimage import io
%matplotlib inline
import pickle

from aux_fun import *
from p4_fun import *

Training parameters definition:

In [2]:
PARAMS = {
    'use_spatial': True,
    'spatial_cspace': 'YCrCb',
    'spatial_resize': 32,
    
    'use_hist': True,
    'hist_cspace': 'YCrCb',
    'hist_nbins': 32,
    
    'use_hog': True,
    'hog_cspace': 'YCrCb',
    'channels_for_hog': [0,1,2],
    'hog_orientations': 9,
    'hog_pixels_per_cell': 16,
    'hog_cells_per_block': 2,
    
    'cells_per_step_overlap': 1
}

## Building the training set

In [3]:
car_image_fnames = glob.glob('data/vehicles/*/*.png')
not_car_image_fnames = glob.glob('data/non-vehicles/*/*.png')

In [4]:
X, y = build_dataset(
    car_image_fnames, not_car_image_fnames, 
    hog_orientations=PARAMS['hog_orientations'], 
    hog_pixels_per_cell=PARAMS['hog_pixels_per_cell'], 
    hog_cells_per_block=PARAMS['hog_cells_per_block'],
    hist_nbins=PARAMS['hist_nbins'],
    spatial_resize=PARAMS['spatial_resize'],
    channels_for_hog=PARAMS['channels_for_hog'],
    use_hog=PARAMS['use_hog'],
    hog_cspace=PARAMS['hog_cspace'],
    use_spatial=PARAMS['use_spatial'],
    spatial_cspace=PARAMS['spatial_cspace'],
    use_hist=PARAMS['use_hist'],
    hist_cspace=PARAMS['hist_cspace']
)
print('Dataset size: ', X.shape)

Dataset size:  (17760, 4140)


## Training pipeline

In [5]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

scaled_clf = make_pipeline(
    StandardScaler(),
    LinearSVC(
        C=10
    )
)

In [6]:
scaled_clf.fit(X,y)

Pipeline(steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('linearsvc', LinearSVC(C=10, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

## Lane pipeline

In [7]:
class Line():
    def __init__(self):
        self.previous_fit_coeffs = None

In [8]:
def lane_detection_pl(original_img_rgb, thr_pipeline, l_lane, r_lane, 
                      camera_params, src_vertices, dst_vertices):
    
    undist_img = undistort(original_img_rgb, camera_params)
    binary_img = thr_pipeline(undist_img)
    binary_warped = warp_image(binary_img, src_vertices, dst_vertices)
    
    if (l_lane.previous_fit_coeffs is None) or (l_lane.previous_fit_coeffs is None):
        leftx, lefty, rightx, righty, _ = find_lanes_sliding_window_hist(binary_warped, get_viz=False)
    else:
        leftx, lefty, rightx, righty, _ = find_lanes_near_previous(
            binary_warped,
            l_lane.previous_fit_coeffs,
            r_lane.previous_fit_coeffs,
            get_viz=False)
        
    left_fit, right_fit = get_lane_fit_coeffs(leftx, lefty, rightx, righty)
    l_lane.previous_fit_coeffs = left_fit
    r_lane.previous_fit_coeffs = right_fit
    
    y_eval = binary_warped.shape[0]
    r_left = calculate_radius_in_meters(y_eval, left_fit, 3.7/700, 30/720)
    r_right = calculate_radius_in_meters(y_eval, right_fit, 3.7/700, 30/720)
    
    offset = calculate_offset_in_meters(binary_warped, left_fit, right_fit, 3.7/700)
    
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    res = print_summary_on_original_image(
        undist_img, binary_warped,
        left_fitx, right_fitx, ploty,
        leftx, lefty, rightx, righty,
        r_left, r_right, offset,
        src_vertices, dst_vertices
    )
    return res

In [9]:
camera_params = pickle.load(open('challenge/camera_params.p', 'rb'))
src_vertices = pickle.load(open('challenge/src_vertices.p', 'rb'))
dst_vertices = pickle.load(open('challenge/dst_vertices.p', 'rb'))

In [10]:
lane_process = lambda img, l_lane, r_lane: lane_detection_pl(
    img, thresholding_pipeline, l_lane, r_lane, camera_params, src_vertices, dst_vertices
)

## Prediction pipeline

In [11]:
# wrapper for the find_cars() method to be easier to use
detect_for_scale = lambda img, ystart, ystop, scale: find_cars(
    img, 
    ystart=ystart, ystop=ystop, 
    scale=scale,
    cells_per_step_overlap=PARAMS['cells_per_step_overlap'],
    clf_pl=scaled_clf, 
    use_hog=PARAMS['use_hog'], hog_cspace=PARAMS['hog_cspace'], 
    channels_for_hog=PARAMS['channels_for_hog'],
    orient=PARAMS['hog_orientations'], 
    pix_per_cell=PARAMS['hog_pixels_per_cell'], 
    cell_per_block=PARAMS['hog_cells_per_block'],
    use_spatial=PARAMS['use_spatial'], spatial_cspace=PARAMS['spatial_cspace'], 
    spatial_size=PARAMS['spatial_resize'], 
    use_hist=PARAMS['use_hist'], hist_cspace=PARAMS['hist_cspace'], 
    hist_bins=PARAMS['hist_nbins']
)

In [12]:
from scipy.ndimage.measurements import label
class VehicleTracker:
    def __init__(self, frame_size, new_frame_factor=0.5, heatmap_threshold=2):
        self.new_frame_factor = new_frame_factor
        self.heatmap_threshold = heatmap_threshold
        self.heatmap = np.zeros(frame_size)
        self.l_lane = Line()
        self.r_lane = Line()
        
    def process_frame(self, frame):
        
        # search for detections over different scaled windows and store the 
        # positive bounding boxes in the hot_windows list
        hot_windows = list()
        hot_windows += detect_for_scale(frame, ystart=400, ystop=500, scale=1.0)
        hot_windows += detect_for_scale(frame, ystart=400, ystop=600, scale=2.0)
        hot_windows += detect_for_scale(frame, ystart=400, ystop=650, scale=3.0)
        
        # build heatmap based on the multi-scale detections
        frame_heat = np.zeros_like(frame[:,:,0]).astype(np.float)
        frame_heat = add_heat(frame_heat, hot_windows)
        
        # apply weighted average between the current heatmap and the averaged over the 
        # previous steps
        self.heatmap = self.new_frame_factor * frame_heat + (1 - self.new_frame_factor) * self.heatmap
        
        # threshold the resulting heatmap to highlight high-confidence detections
        thresholded_heatmap = apply_threshold(self.heatmap, threshold=self.heatmap_threshold)
        
        # define tight bounding boxes for the high-confidence detections
        labels = label(thresholded_heatmap)
        
        # print lane information
        annotated_lane_img = lane_process(frame, self.l_lane, self.r_lane)
        
        # draw the boxes 
        proccessed_frame = draw_labeled_bboxes(annotated_lane_img, labels)
        
        return proccessed_frame

## Video generation

In [13]:
from moviepy.editor import VideoFileClip

In [14]:
build_video = True
if build_video:
    vehicleTracker = VehicleTracker(
        frame_size=(720, 1280), 
        new_frame_factor=0.5, 
        heatmap_threshold=2
    )
    output = 'challenge/challenge_solution.mp4'
    clip = VideoFileClip('project_video.mp4')
    output_clip = clip.fl_image(vehicleTracker.process_frame)
    output_clip.write_videofile(output, audio=False)

[MoviePy] >>>> Building video challenge/challenge_solution.mp4
[MoviePy] Writing video challenge/challenge_solution.mp4


100%|█████████▉| 1260/1261 [06:18<00:00,  3.33it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: challenge/challenge_solution.mp4 

